In [ ]:
# обеспечиваем совместимость с Python 2 и 3
from __future__ import (absolute_import, division, print_function, unicode_literals)

# отключаем предупреждения дистрибутива Anaconda
import warnings
warnings.simplefilter('ignore')

**Описание датасета:**   
[Датасет вакансий с платформы HH.ru на Kaggle.com](https://www.kaggle.com/datasets/pavfedotov/heaadhunter-vacancies?resource=download&select=df2021-08-03.csv) 

Нужный файл - df2021-08-03.csv.

- *vacancy* - наименование вакансии
- *url* - ссылка на вакансию
- *created* - дата и время создания
- *has_test* - наличие тестового задания в вакансии
- *salary_from* - нижняя граница значения заработной платы
- *salary_to* - верхняя граница значения заработной платы
- *currency* - валюта заработной платы
- *experience* - требуемый опыт
- *schedule* - тип рабочего графика
- *skills* - требуемые навыки
- *employer* - наименование работодателя
- *area* - наименование города
- *description* - описание вакансии

#### Загрузите датасет

In [ ]:
#Необходимые библиотеки
import pandas as pd
import matplotlib.pyplot as plt
import re
import plotly.express as px

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/df2021-08-03.csv')


In [ ]:
df.head()

,vacancy,url,created,has_test,salary_from,salary_to,currency,experience,schedule,skills,employer,area,description
0,Backend/Full-stack developer (python),https://hh.ru/applicant/vacancy_response?vacan...,2021-08-01T13:02:48+0300,False,120000.0,NaN,RUR,От 1 года до 3 лет,Гибкий график,Python;PostgreSQL;Linux;Flask;,ATI.SU,Санкт-Петербург,Привет! Мы ATI.SU ― крупнейшая в России компан...
1,Бэкенд-разработчик (Python) / Middle Python / ...,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-09T08:13:01+0300,False,150000.0,220000.0,RUR,От 3 до 6 лет,Удаленная работа,Python;Git;MongoDB;Redis;Design Patterns;Flask;,"ЮТэйр, Авиакомпания",Киров (Кировская область),Utair - российская авиакомпания. Мы летаем по ...
2,Fullstack Middle Python (Django) / VueJS Dev,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-21T15:24:16+0300,False,NaN,NaN,NaN,От 1 года до 3 лет,Полный день,Python;Git;Django Framework;PostgreSQL;Linux;V...,КРОК,Москва,"В команду, которая занимается разработкой инст..."
3,Python Developer,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-28T12:57:57+0300,False,NaN,NaN,NaN,От 3 до 6 лет,Удаленная работа,Python;Django Framework;CSS;,Mad Devs,Краснодар,Mad Devs - full stack команда для разработки и...
4,QA Automation (Python),https://hh.ru/applicant/vacancy_response?vacan...,2021-07-31T17:10:49+0300,False,NaN,NaN,NaN,От 1 года до 3 лет,Удаленная работа,Python;Pytest;Ansible;Selenium;,Аренадата Софтвер,Москва,"Arenadata — динамично развивающаяся компания, ..."


In [ ]:
#Поиск пустых значений
df.isnull().sum()

vacancy           0
url               0
created           0
has_test          0
salary_from    1162
salary_to      1267
currency       1087
experience        0
schedule          0
skills          210
employer          0
area              0
description       0
dtype: int64

In [ ]:
df['currency'].unique()

array(['RUR', nan, 'EUR', 'USD'], dtype=object)

### Практические задания:

#### 1. Изучите методы для сбора данных курсов валют в [API Центрального банка Российской Федерации](https://www.cbr.ru/development/SXML/) . Используя запросы к API, приведите значения заработной платы к рублям одним из вариантов: 1) простой вариант - на текущую дату  2) вариант посложнее - на дату создания (created).

In [ ]:
import requests
import xml.etree.ElementTree as ET
#Отправляем запрос на получение курса валюты на текущую дату
response = requests.get('https://www.cbr.ru/scripts/XML_daily.asp')
root = ET.fromstring(response.text)

#Ищем курс евро и доллара в полученном XML-документе, так как в датасете в столбце с валютой 3 уникальных значения: рубли, доллары и евро
for valute in root.findall('Valute'):
    if valute.find('CharCode').text == 'EUR':
        rate_eur = float(valute.find('Value').text.replace(',', '.')) #Сохраняем это значение в отдельную переменную
    if valute.find('CharCode').text == 'USD':
        rate_usd = float(valute.find('Value').text.replace(',', '.'))

#Ищем строки, в которых значение валюты EUR, перевод зарплаты из евро в рубли и перезапись в столбцах 'salary_from' и 'salary_to'
df.loc[df['currency'] == 'EUR', ['salary_from', 'salary_to']] = df.loc[df['currency'] == 'EUR', ['salary_from', 'salary_to']] * rate_eur
df.loc[df['currency'] == 'EUR', ['salary_from', 'salary_to']] = df.loc[df['currency'] == 'EUR', ['salary_from', 'salary_to']].round(1)
#Замена наименования валюты EUR на RUR
df.loc[df['currency'] == 'EUR', 'currency'] = 'RUR'
#Аналогичная операция с USD
df.loc[df['currency'] == 'USD', ['salary_from', 'salary_to']] = df.loc[df['currency'] == 'USD', ['salary_from', 'salary_to']] * rate_usd
df.loc[df['currency'] == 'USD', ['salary_from', 'salary_to']] = df.loc[df['currency'] == 'USD', ['salary_from', 'salary_to']].round(1)
df.loc[df['currency'] == 'USD', 'currency'] = 'RUR'


In [ ]:
df.loc[df['currency'] =='EUR'] #Доказательство, что строк, где бы была валюта евро, не осталось

,vacancy,url,created,has_test,salary_from,salary_to,currency,experience,schedule,skills,employer,area,description


In [ ]:
df['currency'].unique() #Остались только рубли

array(['RUR', nan], dtype=object)

#### 2. Найдите вакансию с самой высокой заработной платой, где подойдет 4 года опыта работы и не нужно проходить тестовое задание.

In [ ]:
df['experience'].unique()


array(['От 1 года до 3 лет', 'От 3 до 6 лет', 'Нет опыта', 'Более 6 лет'],
      dtype=object)

In [ ]:
#Используем регулярные выражения для извлечения чисел из столбца experience, занесем числовые значения в отдельный столбец experience_years
#будем заносить первое число, так как, например, с опытом от 1 до 3 лет, обязателен опыт именно больше 1 года, верхний предел обычно менее важен
#то есть с четырехлетним опытом могут взять на работу, где требуется опыт от 1 до 3 лет, но не наоборот

def experience(exp): #функция для извлечениия чисел
    pattern = r'\d+'
    matches = re.findall(pattern, exp)
    if len(matches) == 1:
        return int(matches[0])
    elif len(matches) == 2:
        return int(matches[0])
    else:
        return 0 #если нет опыта записываем 0
df['experience_years'] = df['experience'].apply(experience)
df.sample(3)

,vacancy,url,created,has_test,salary_from,salary_to,currency,experience,schedule,skills,employer,area,description,experience_years
1011,Python Developer,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-08T20:26:22+0300,False,NaN,NaN,NaN,От 3 до 6 лет,Гибкий график,Python;Английский язык;Linux;SQL;,Auriga,Ростов-на-Дону,АУРИГА — российско-американская компания с 30-...,3
1542,Backend Python Developer (удалённо),https://hh.ru/applicant/vacancy_response?vacan...,2021-07-26T10:19:29+0300,False,300000.0,400000.0,RUR,От 3 до 6 лет,Удаленная работа,Python;Linux;Bash;Cloud;VoIP;SIP;,IT_Selection,Москва,"Уважаемые соискатели, наш клиент, зарубежная к...",3
951,Бэкенд-разработчик/Python/Middle Python/ Pytho...,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-06T15:02:17+0300,False,150000.0,220000.0,RUR,От 3 до 6 лет,Удаленная работа,Python;Git;MongoDB;Redis;Design Patterns;Flask;,"ЮТэйр, Авиакомпания",Тверь,Utair - российская авиакомпания. Мы летаем по...,3


In [ ]:
#Выбираем вакансии, где требуется опыт работы менее 4 лет и не нужно проходить тестовое задание
df2 = df[(df['experience_years'] <= 4) & (df['has_test'] == False)]

#Ищем вакансию с самой высокой зарплатой, но так как в столбцах salary_from и salary_to могут быть пустые значения, найдем для каждого
highest_salary_vacancy_to = df2.loc[df2['salary_to'].idxmax()]
highest_salary_vacancy_from = df2.loc[df2['salary_from'].idxmax()]
print(highest_salary_vacancy_to, '\n')
print(highest_salary_vacancy_from)


#Так получилось, что у второй вакансии нет верхнего предела, а стартовая зарплата у нее выше, поэтому нельзя ее исключить

vacancy                                  Back-end Developer (Python3)
url                 https://hh.ru/applicant/vacancy_response?vacan...
created                                      2021-08-02T13:21:00+0300
has_test                                                        False
salary_from                                                  320000.0
salary_to                                                    550000.0
currency                                                          RUR
experience                                         От 1 года до 3 лет
schedule                                                Гибкий график
skills              Git;Linux;Python;PostgreSQL;Django Framework;R...
employer                                                       Фортин
area                                                           Москва
description         Что для нас важно:  Опыт разработки на Python3...
experience_years                                                    1
Name: 959, dtype: ob

#### 3. Сделайте рейтинг (топ-30) навыков по всем вакансиям.

In [ ]:
df['skills'].unique()

array(['Python;PostgreSQL;Linux;Flask;',
       'Python;Git;MongoDB;Redis;Design Patterns;Flask;',
       'Python;Git;Django Framework;PostgreSQL;Linux;Vue.js;', ...,
       'Python;Git;Linux;C++;x86;Windows;Unix;Virtualization;',
       'Git;PostgreSQL;Python;Blockchain;Crypto;React (hooks);TypeScript;Redux;Agile Project Management;React.js;',
       'Java Servlets;Java;JavaScript;Spring Framework;Git;JDBC;HTML5;Apache Tomcat;Atlassian Jira;SQL;Nginx;ООП;AngularJS;Eclipse;Kafka;'],
      dtype=object)

In [ ]:
#Создаем пустой словарь для хранения навыков и их количества упоминаний
skills_count = {}

#Проходим по всем вакансиям
for index, row in df.iterrows():
    #Извлекаем из строки требуемые навыки и разбиваем их по ';' на отдельные элементы
    skills = str(row['skills']).split(';')
    
    #Увеличиваем счетчик упоминаний для каждого навыка
    for skill in skills:
        if skill: #Проверка на пустую строку
            if skill in skills_count:
                skills_count[skill] += 1
            else:
                skills_count[skill] = 1

#Сортируем словарь по убыванию количества упоминаний и выводим топ-30 навыков
top_skills = sorted(skills_count.items(), key=lambda x: x[1], reverse=True)[:30]
top_skills

#Можно более красивым выводом
#for skill, count in top_skills:
    #print(f'{skill}: {count}')


[('Python', 1442),
 ('Git', 586),
 ('SQL', 501),
 ('PostgreSQL', 499),
 ('Django Framework', 467),
 ('Linux', 463),
 ('nan', 210),
 ('Flask', 177),
 ('Docker', 172),
 ('JavaScript', 149),
 ('Английский язык', 141),
 ('MySQL', 128),
 ('ООП', 120),
 ('MongoDB', 113),
 ('Redis', 107),
 ('REST', 70),
 ('RabbitMQ', 68),
 ('Django', 62),
 ('CSS', 56),
 ('HTML', 56),
 ('Atlassian Jira', 52),
 ('C++', 50),
 ('API', 49),
 ('MS SQL', 43),
 ('Java', 43),
 ('Базы данных', 41),
 ('Работа в команде', 39),
 ('AWS', 36),
 ('Pytest', 35),
 ('Celery', 35)]

#### 4. Cоставьте топ-10 городов с наибольшей средней заработной платой по вакансиям тестировщика (QA).

In [ ]:
#Используем метод str.contains() для поиска вакансий, содержащих слово QA, однако еще можно использовать регулярные выражения
#pattern = re.compile(r'\bQA\b', flags=re.IGNORECASE) игнорируя регистр
#df4 = df[df['vacancy'].str.contains(pattern)]

df4 = df[df['vacancy'].str.contains('QA')]
print('Всего городов, где представлены вакансии тестировщиков:', df4['area'].nunique(), '\n')

#Заполняем пропущенные значения в столбцах salary_from значением из столбца salary_to и salary_to значением из столбца salary_from
df4['salary_from'].fillna(df4['salary_to'], inplace=True)
df4['salary_to'].fillna(df4['salary_from'], inplace=True)

#Функция расчета средней зарплаты
def calculate_mean_salary(row):
    return (row['salary_from'] + row['salary_to']) / 2

#Применяем функцию к каждой строке и создаем новый столбец со значениями средней зарплаты
df4['mean_salary'] = df4.apply(calculate_mean_salary, axis=1)

#Группируем данные по городам и считаем среднее значение зарплаты в каждом городе
df4_grouped = df4.groupby('area')['mean_salary'].mean()

#Сортируем данные по убыванию средней зарплаты и выбираем топ-10 городов
top_cities = df4_grouped.sort_values(ascending=False)[:10]
top_cities

Всего городов, где представлены вакансии тестировщиков: 7 



area
Тамбов             200000.000000
Москва             190555.555556
Санкт-Петербург    180555.555556
Казань              90000.000000
Новосибирск                  NaN
Самара                       NaN
Саратов                      NaN
Name: mean_salary, dtype: float64

#### 5. Покажите динамику количества вакансии по месяцам даты создания.

In [ ]:
df5 = df.copy() #Сделаем копию на всякий случай

#Преобразуем столбец 'created' в формат даты
df5['created'] = pd.to_datetime(df5['created'])

#Создаем столбец с месяцем создания вакансии
df5['month_created'] = df5['created'].dt.month

#Группируем по месяцу создания и считаем количество вакансий в каждом месяце
vacancies_by_month = df5.groupby('month_created').size().reset_index(name='count')

vacancies_by_month

,month_created,count
0,7,1239
1,8,461


In [ ]:
#Строим график для наглядности
fig = px.bar(vacancies_by_month, x='month_created', y='count')
fig.update_layout(title='Динамика количества вакансий по месяцам даты создания')
fig.show()

#### 6. Напишите функцию для поиска вакансий по переданным в нее значениям заработной платы, наличия тестового задания, наличия заданного текста в наименовании вакансии, наличия заданного навыка. В результате выведите количество найденных вакансий, среднюю заработную плату по найденным вакансиям и список ссылок найденных вакансий. 

In [ ]:
#Делаем df таким же, как df4, так как в нем содержится столбец со средней зарплатой
df = df4
def search_vacancies(salary_from=None, salary_to=None, has_test=None, text_in_vacancy=None, text_in_skill=None):
    #Фильтруем по переданным параметрам
    filtered_df = df.copy() #Скопируем на всякий случай

    if salary_from is not None:
        filtered_df = filtered_df[filtered_df['salary_from'] >= salary_from] 
    
    if salary_to is not None:
        filtered_df = filtered_df[filtered_df['salary_to'] <= salary_to]
    
    if has_test is not None:
        filtered_df = filtered_df[filtered_df['has_test'] == has_test]
    
    if text_in_vacancy is not None:
        #Используя, регулярные выражения делаем поиск указанного слова в названии самой вакансии
        pattern = re.compile(text_in_vacancy, flags=re.IGNORECASE)
        filtered_df = filtered_df[filtered_df['vacancy'].str.contains(pattern)]
    if text_in_skill is not None:
        #Используя, регулярные выражения делаем поиск указанного слова в скиллах
        pattern = re.compile(text_in_skill, flags=re.IGNORECASE)
        filtered_df = filtered_df[filtered_df['skills'].str.contains(pattern)]
    
    #Подсчитываем количество найденных вакансий и среднюю зарплату
    num_vacancies = len(filtered_df)
    mean_salary = filtered_df['mean_salary'].mean()
    
    #Получаем список ссылок на найденные вакансии
    vacancy_links = filtered_df['url'].tolist()
    
    return num_vacancies, mean_salary, vacancy_links

result = search_vacancies(100000, 220000, False, 'QA', 'Python')
for i in result:
  print(i)

6
146666.66666666666
['https://hh.ru/applicant/vacancy_response?vacancyId=46582713', 'https://hh.ru/applicant/vacancy_response?vacancyId=46693471', 'https://hh.ru/applicant/vacancy_response?vacancyId=46550714', 'https://hh.ru/applicant/vacancy_response?vacancyId=45062852', 'https://hh.ru/applicant/vacancy_response?vacancyId=46212451', 'https://hh.ru/applicant/vacancy_response?vacancyId=46529767']


#### *Необязательно*. Усложненный вариант - сделать атрибуты функции опциональными, чтобы иметь возможность фильтрации вакансий по любой комбинации условий.

In [ ]:
#См. предыдущее задание
result2 = search_vacancies(salary_to=500000, text_in_skill='SQL')
for i in result2:
  print(i)


8
148750.0
['https://hh.ru/applicant/vacancy_response?vacancyId=45783106', 'https://hh.ru/applicant/vacancy_response?vacancyId=46703958', 'https://hh.ru/applicant/vacancy_response?vacancyId=46582713', 'https://hh.ru/applicant/vacancy_response?vacancyId=43954953', 'https://hh.ru/applicant/vacancy_response?vacancyId=46550714', 'https://hh.ru/applicant/vacancy_response?vacancyId=44868124', 'https://hh.ru/applicant/vacancy_response?vacancyId=46212451', 'https://hh.ru/applicant/vacancy_response?vacancyId=46529767']
